# 🏥 Medicine Demand Forecasting - Quick Training
## Using synthetic_dispense_records_280.csv

**Fast-track training for Barangay Health Center**

This notebook will:
1. ✅ Load your CSV data
2. ✅ Train Gradient Boosting models
3. ✅ Generate forecasts (monthly, quarterly, seasonal)
4. ✅ Export models for VPS deployment

⏱️ **Estimated time:** 10-15 minutes

---

## 📦 Step 1: Install Required Packages

In [ ]:
%%capture
# Install required packages (runs silently)
!pip install pandas numpy scikit-learn joblib matplotlib seaborn plotly

print("✅ Packages installed!")

## 📚 Step 2: Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import warnings
import os
import joblib
import calendar
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import matplotlib.pyplot as plt
import seaborn as sns

# Import display for Jupyter/Colab
from IPython.display import display

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

print("✅ Libraries imported!")
print(f"📅 Today: {datetime.now().strftime('%Y-%m-%d')}")

## 📤 Step 3: Upload Your CSV File

**Click "Choose Files" and upload:** `synthetic_dispense_records_280.csv`

In [ ]:
from google.colab import files

print("📤 Please upload: synthetic_dispense_records_280.csv")
print("")
uploaded = files.upload()

# Get the filename
csv_filename = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {csv_filename}")
print(f"   File size: {len(uploaded[csv_filename]) / 1024:.1f} KB")

## 📊 Step 4: Load and Explore Data

In [ ]:
# Load CSV
df_raw = pd.read_csv(csv_filename)

print("="*80)
print("DATA OVERVIEW")
print("="*80)
print(f"\n📋 Total Records: {len(df_raw):,}")
print(f"💊 Unique Medicines: {df_raw['med_name'].nunique()}")
print(f"📅 Date Range: {df_raw['date_given'].min()} to {df_raw['date_given'].max()}")

print("\n📊 Sample Data:")
print(df_raw.head(10).to_string())

print("\n📈 Medicine Distribution:")
med_counts = df_raw['med_name'].value_counts()
print(med_counts)

print("\n💊 Medicines in dataset:")
print(df_raw['med_name'].unique())

## 📈 Step 5: Visualize Data

In [ ]:
# Convert date column to datetime FIRST
df_raw['date_given'] = pd.to_datetime(df_raw['date_given'])

# Total quantity by medicine
print("📊 Creating visualizations...\n")

plt.figure(figsize=(14, 6))
med_totals = df_raw.groupby('med_name')['quantity_given'].sum().sort_values(ascending=False)

# Convert to lists for plotting (avoid any index issues)
med_names = med_totals.index.tolist()
med_values = med_totals.values.tolist()

plt.barh(med_names, med_values, color='steelblue')
plt.title('Total Dispensing by Medicine', fontsize=16, fontweight='bold')
plt.xlabel('Total Quantity', fontsize=12)
plt.ylabel('Medicine', fontsize=12)
plt.tight_layout()
plt.show()

# Monthly trend for top medicine - FULLY FIXED VERSION
top_med = med_totals.index[0]
top_med_data = df_raw[df_raw['med_name'] == top_med].copy()

# Create year-month column as string first
top_med_data['year_month_str'] = top_med_data['date_given'].dt.strftime('%Y-%m')

# Aggregate by month
monthly_agg = top_med_data.groupby('year_month_str')['quantity_given'].sum().reset_index()
monthly_agg['date'] = pd.to_datetime(monthly_agg['year_month_str'] + '-01')
monthly_agg = monthly_agg.sort_values('date')

# Extract numeric lists for plotting
dates = monthly_agg['date'].tolist()
quantities = monthly_agg['quantity_given'].tolist()

plt.figure(figsize=(14, 6))
plt.plot(dates, quantities, marker='o', linewidth=2, markersize=8, color='orangered')
plt.title(f'Monthly Trend: {top_med}', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=12)
plt.ylabel('Quantity', fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print(f"\n✅ Visualizations complete for {len(med_totals)} medicines")
print(f"📊 Top medicine: {top_med} with {med_totals.iloc[0]:.0f} total units dispensed")

## 🔧 Step 6: Prepare Data for Training

In [ ]:
print("🔧 Preparing data for training...\n")

# Configuration
SEASONS = {
    'Dry Season (Tag-init)': [3, 4, 5],
    'Wet Season (Tag-ulan)': [6, 7, 8, 9],
    'Cool Dry (Amihan)': [12, 1, 2],
    'Transition': [10, 11]
}

def get_season(month):
    for season, months in SEASONS.items():
        if month in months:
            return season
    return 'Transition'

# Convert dates
df = df_raw.copy()
df['date_given'] = pd.to_datetime(df['date_given'])
df['period'] = df['date_given'].dt.to_period('M')

# Get all medicines and date range
all_medicines = df['med_name'].unique().tolist()
min_period = df['period'].min()
max_period = df['period'].max()
full_period_range = pd.period_range(start=min_period, end=max_period, freq='M')

print(f"📅 Training period: {min_period} to {max_period} ({len(full_period_range)} months)")
print(f"💊 Training for {len(all_medicines)} medicines")

# Create continuous time series
multi_index = pd.MultiIndex.from_product(
    [all_medicines, full_period_range],
    names=['med_name', 'period']
)
monthly_template = pd.DataFrame(index=multi_index).reset_index()
monthly_template['date_start'] = monthly_template['period'].apply(lambda x: x.start_time)

# Aggregate monthly usage
monthly_usage = df.groupby(['med_name', 'period'])['quantity_given'].sum().reset_index()

# Merge and fill gaps
monthly_df = monthly_template.merge(monthly_usage, on=['med_name', 'period'], how='left')
monthly_df['total_quantity'] = monthly_df['quantity_given'].fillna(0)

print("✅ Continuous time series created")
print(f"   Total records: {len(monthly_df):,}")

## 🎨 Step 7: Feature Engineering

In [ ]:
print("🎨 Engineering features...\n")

# Time features
monthly_df['month_of_year'] = monthly_df['date_start'].dt.month
monthly_df['quarter'] = monthly_df['date_start'].dt.quarter
monthly_df['days_in_month'] = monthly_df['date_start'].dt.days_in_month
monthly_df['season'] = monthly_df['month_of_year'].apply(get_season)

# Cyclical encoding
monthly_df['month_sin'] = np.sin(2 * np.pi * monthly_df['month_of_year'] / 12)
monthly_df['month_cos'] = np.cos(2 * np.pi * monthly_df['month_of_year'] / 12)

# Lag features
for lag in [1, 2, 3, 6, 12]:
    monthly_df[f'lag_{lag}'] = monthly_df.groupby('med_name')['total_quantity'].shift(lag)

# Rolling statistics
for window in [3, 6, 12]:
    monthly_df[f'rolling_mean_{window}'] = monthly_df.groupby('med_name')['total_quantity'].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )
    monthly_df[f'rolling_std_{window}'] = monthly_df.groupby('med_name')['total_quantity'].transform(
        lambda x: x.rolling(window=window, min_periods=1).std().fillna(0)
    )

# Time index (trend)
monthly_df['time_index'] = monthly_df.groupby('med_name').cumcount()

# Holiday features (from CSV if available)
if 'is_national_holiday' in df.columns:
    holiday_data = df.groupby('period')['is_national_holiday'].sum().reset_index()
    holiday_data.columns = ['period', 'total_holidays']
    monthly_df = monthly_df.merge(holiday_data, on='period', how='left')
    monthly_df['total_holidays'] = monthly_df['total_holidays'].fillna(0)
    monthly_df['holiday_ratio'] = monthly_df['total_holidays'] / monthly_df['days_in_month']
    print("   ✅ Holiday features added")
else:
    monthly_df['total_holidays'] = 0
    monthly_df['holiday_ratio'] = 0
    print("   ⚠️ No holiday data - using zeros")

# Medicine encoding (simple numeric)
med_mapping = {med: idx for idx, med in enumerate(all_medicines)}
monthly_df['med_encoded'] = monthly_df['med_name'].map(med_mapping)

print("\n✅ Feature engineering complete!")
print(f"   Features created: {len(monthly_df.columns)}")

print("\n📊 Sample of engineered features:")
print(monthly_df.head(10).to_string())

## 🎯 Step 8: Train Gradient Boosting Models

In [ ]:
print("="*80)
print("TRAINING GRADIENT BOOSTING MODELS")
print("="*80)
print()

# Create directories
os.makedirs('models', exist_ok=True)
os.makedirs('results', exist_ok=True)

# Feature columns
feature_cols = [
    'month_of_year', 'quarter', 'month_sin', 'month_cos',
    'lag_1', 'lag_2', 'lag_3', 'lag_6', 'lag_12',
    'rolling_mean_3', 'rolling_mean_6', 'rolling_mean_12',
    'rolling_std_3', 'rolling_std_6', 'rolling_std_12',
    'time_index', 'holiday_ratio', 'days_in_month',
    'med_encoded'
]

models = {}
metrics = []
trained = 0
skipped = 0

for i, med in enumerate(all_medicines, 1):
    # Get medicine data
    med_data = monthly_df[monthly_df['med_name'] == med].copy()
    
    # Remove NaN in critical features
    med_data_clean = med_data.dropna(subset=['lag_1', 'lag_2', 'lag_3'])
    
    if len(med_data_clean) < 6:
        skipped += 1
        print(f"   ⏭️  Skipped {med} (insufficient data: {len(med_data_clean)} months)")
        continue
    
    # Prepare data
    X = med_data_clean[feature_cols].values
    y = med_data_clean['total_quantity'].values
    
    # Train-test split
    if len(X) >= 12:
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, shuffle=False
        )
    else:
        X_train, y_train = X, y
        X_test, y_test = None, None
    
    # Train model
    model = GradientBoostingRegressor(
        n_estimators=150,
        learning_rate=0.1,
        max_depth=4,
        min_samples_split=4,
        min_samples_leaf=2,
        subsample=0.8,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    # Evaluate
    if X_test is not None:
        y_pred = model.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        metrics.append({
            'medicine': med,
            'train_samples': len(X_train),
            'test_samples': len(X_test),
            'mae': mae,
            'rmse': rmse,
            'r2': r2
        })
    else:
        mae, rmse, r2 = None, None, None
    
    # Store model
    models[med] = {
        'model': model,
        'last_data': med_data.iloc[-1].to_dict(),
        'mae': mae,
        'r2': r2
    }
    
    # Save model
    model_path = f"models/{med.replace(' ', '_')}_enhanced_gbr.joblib"
    joblib.dump(model, model_path)
    
    trained += 1
    print(f"   ✅ Trained: {med} (MAE: {mae:.2f if mae else 'N/A'}, R²: {r2:.3f if r2 else 'N/A'})")

print(f"\n{'='*80}")
print(f"✅ Training Complete!")
print(f"   Models trained: {trained}")
print(f"   Skipped: {skipped}")
print(f"{'='*80}")

if metrics:
    df_metrics = pd.DataFrame(metrics)
    print("\n📊 Average Performance:")
    print(f"   MAE:  {df_metrics['mae'].mean():.2f}")
    print(f"   RMSE: {df_metrics['rmse'].mean():.2f}")
    print(f"   R²:   {df_metrics['r2'].mean():.3f}")
    
    print("\n🏆 Model Performance Details:")
    print(df_metrics.to_string())

## 🔮 Step 9: Generate Forecasts

In [ ]:
print("🔮 Generating forecasts...\n")

def generate_future_features(models, med_name, months_ahead=12):
    """Generate future feature vectors."""
    last_data = models[med_name]['last_data']
    last_date = pd.to_datetime(last_data['date_start'])
    
    future_features = []
    
    for i in range(1, months_ahead + 1):
        future_date = last_date + relativedelta(months=i)
        
        month = future_date.month
        quarter = (month - 1) // 3 + 1
        month_sin = np.sin(2 * np.pi * month / 12)
        month_cos = np.cos(2 * np.pi * month / 12)
        days_in_month = calendar.monthrange(future_date.year, future_date.month)[1]
        time_index = last_data['time_index'] + i
        
        # Lags (simplified)
        if i == 1:
            lag_1 = last_data['total_quantity']
            lag_2 = last_data.get('lag_1', lag_1)
            lag_3 = last_data.get('lag_2', lag_1)
            lag_6 = last_data.get('lag_5', lag_1)
            lag_12 = last_data.get('lag_11', lag_1)
        else:
            lag_1 = future_features[i-2][4] if i > 1 else last_data['total_quantity']
            lag_2 = future_features[i-3][4] if i > 2 else last_data.get('lag_1', lag_1)
            lag_3 = future_features[i-4][4] if i > 3 else last_data.get('lag_2', lag_1)
            lag_6 = last_data.get('lag_5', lag_1)
            lag_12 = last_data.get('lag_11', lag_1)
        
        # Rolling stats (approximate)
        rolling_mean_3 = last_data.get('rolling_mean_3', lag_1)
        rolling_mean_6 = last_data.get('rolling_mean_6', lag_1)
        rolling_mean_12 = last_data.get('rolling_mean_12', lag_1)
        rolling_std_3 = last_data.get('rolling_std_3', 0)
        rolling_std_6 = last_data.get('rolling_std_6', 0)
        rolling_std_12 = last_data.get('rolling_std_12', 0)
        
        holiday_ratio = last_data.get('holiday_ratio', 0.1)
        med_encoded = last_data.get('med_encoded', 0)
        
        features = [
            month, quarter, month_sin, month_cos,
            lag_1, lag_2, lag_3, lag_6, lag_12,
            rolling_mean_3, rolling_mean_6, rolling_mean_12,
            rolling_std_3, rolling_std_6, rolling_std_12,
            time_index, holiday_ratio, days_in_month,
            med_encoded
        ]
        
        future_features.append(features)
    
    return np.array(future_features)

def calculate_seasonal(monthly_preds):
    """Calculate seasonal predictions."""
    current_month = datetime.now().month
    seasonal = {}
    
    month_preds = {}
    for i, pred in enumerate(monthly_preds[:12]):
        month = ((current_month + i - 1) % 12) + 1
        month_preds[month] = pred
    
    for season, months in SEASONS.items():
        values = [month_preds.get(m, 0) for m in months if m in month_preds]
        seasonal[season] = round(float(np.mean(values)) if values else 0, 2)
    
    return seasonal

# Generate forecasts for all models
all_forecasts = {}

for med_name in models.keys():
    try:
        X_future = generate_future_features(models, med_name, 12)
        model = models[med_name]['model']
        predictions = model.predict(X_future)
        predictions = np.maximum(predictions, 0)
        
        monthly_preds = [round(float(p), 2) for p in predictions]
        
        # Quarterly
        quarterly_preds = []
        for q in range(0, len(monthly_preds), 3):
            quarterly_preds.append(round(float(np.mean(monthly_preds[q:q+3])), 2))
        
        # Seasonal
        seasonal_preds = calculate_seasonal(monthly_preds)
        
        all_forecasts[med_name] = {
            'monthly': {
                'next_1_month': monthly_preds[0],
                'next_2_months': monthly_preds[1],
                'next_3_months': monthly_preds[2],
                'all_months': monthly_preds
            },
            'quarterly': {
                'next_quarter': quarterly_preds[0],
                'all_quarters': quarterly_preds
            },
            'seasonal': seasonal_preds,
            'model_performance': {
                'mae': models[med_name].get('mae'),
                'r2': models[med_name].get('r2')
            }
        }
        
        print(f"   ✅ {med_name}: Next month = {monthly_preds[0]:.2f}")
    
    except Exception as e:
        print(f"   ❌ Error forecasting {med_name}: {e}")

print(f"\n✅ Generated forecasts for {len(all_forecasts)} medicines")

## 📊 Step 10: Visualize Forecasts

In [ ]:
# Top 5 highest predicted demand
sorted_forecasts = sorted(
    all_forecasts.items(),
    key=lambda x: x[1]['monthly']['next_1_month'],
    reverse=True
)[:5]

print("📈 TOP 5 HIGHEST PREDICTED DEMAND (Next Month):")
print("-" * 60)
for med, data in sorted_forecasts:
    print(f"{med:<40} {data['monthly']['next_1_month']:>6.2f}")
print("-" * 60)

# Visualize top medicine forecast
top_med = sorted_forecasts[0][0]
top_forecast = all_forecasts[top_med]

# Historical data - convert to proper datetime
historical = monthly_df[monthly_df['med_name'] == top_med].copy()
historical = historical.sort_values('date_start')

# Convert to lists for plotting
hist_dates = historical['date_start'].tolist()
hist_quantities = historical['total_quantity'].tolist()

# Future dates
last_date = historical['date_start'].max()
future_dates = [last_date + relativedelta(months=i) for i in range(1, 13)]
future_quantities = top_forecast['monthly']['all_months']

# Plot historical + forecast
plt.figure(figsize=(16, 7))

plt.plot(hist_dates, hist_quantities,
         marker='o', linewidth=2, label='Historical', color='steelblue')

plt.plot(future_dates, future_quantities,
         marker='s', linewidth=2, linestyle='--', label='Forecast', color='orangered')

plt.axvline(x=last_date, color='gray', linestyle=':', linewidth=2, label='Forecast Start')
plt.title(f'Demand Forecast: {top_med}', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Quantity', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Seasonal forecast
seasonal_data = top_forecast['seasonal']
seasons = list(seasonal_data.keys())
values = list(seasonal_data.values())

plt.figure(figsize=(10, 6))
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#FFA07A']
plt.bar(seasons, values, color=colors[:len(seasons)])
plt.title(f'Seasonal Forecast: {top_med}', fontsize=16, fontweight='bold')
plt.xlabel('Season', fontsize=12)
plt.ylabel('Predicted Quantity', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("\n✅ Forecast visualizations complete!")

## 💾 Step 11: Save Results

In [ ]:
print("💾 Saving forecast results...\n")

# Save comprehensive forecast
with open('results/enhanced_forecast_results.json', 'w') as f:
    json.dump(all_forecasts, f, indent=4)
print("✅ Saved: results/enhanced_forecast_results.json")

# Save backward-compatible files
monthly_simple = {med: data['monthly']['next_1_month'] for med, data in all_forecasts.items()}
with open('results/forecast_results.json', 'w') as f:
    json.dump(monthly_simple, f, indent=4)
print("✅ Saved: results/forecast_results.json")

seasonal_forecast = {
    med: {
        'next_month_pred': data['monthly']['next_1_month'],
        'quarter_avg_pred': data['quarterly']['next_quarter']
    }
    for med, data in all_forecasts.items()
}
with open('results/seasonal_forecast.json', 'w') as f:
    json.dump(seasonal_forecast, f, indent=4)
print("✅ Saved: results/seasonal_forecast.json")

# Save metrics
if metrics:
    df_metrics.to_csv('results/model_performance.csv', index=False)
    print("✅ Saved: results/model_performance.csv")

print("\n" + "="*80)
print("📦 EXPORT SUMMARY")
print("="*80)
print(f"\n✅ Models: {len(models)} saved in 'models/'")
print(f"✅ Forecasts: {len(all_forecasts)} saved in 'results/'")
print(f"✅ Total files: {len(os.listdir('models'))} models + {len(os.listdir('results'))} results")
print("\n" + "="*80)

## 📥 Step 12: Download Deployment Package

In [ ]:
import shutil

print("📦 Creating deployment package...\n")

# Create deployment directory
deploy_dir = 'medicine_forecast_deployment'
os.makedirs(deploy_dir, exist_ok=True)

# Copy models and results
shutil.copytree('models', f'{deploy_dir}/models', dirs_exist_ok=True)
shutil.copytree('results', f'{deploy_dir}/forecast_results', dirs_exist_ok=True)

print(f"✅ Copied {len(os.listdir('models'))} models")
print(f"✅ Copied {len(os.listdir('results'))} result files")

# Create README
readme = f"""# Medicine Demand Forecasting - Deployment Package

## Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Contents:
- models/: {len(os.listdir('models'))} trained Gradient Boosting models
- forecast_results/: Forecast predictions (JSON)

## Deployment:
1. Upload to VPS: /home/user/health/
2. Run: ./deploy_models.sh
3. Generate forecasts: python3 forecast_enhanced_gbr.py
4. View dashboard: http://your-server/forecast_dashboard.php

## Trained Medicines:
{chr(10).join('- ' + med for med in all_forecasts.keys())}
"""

with open(f'{deploy_dir}/README.txt', 'w') as f:
    f.write(readme)
print("✅ Created README.txt")

# Create ZIP
print("\n🗜️ Creating ZIP archive...")
shutil.make_archive('medicine_forecast_deployment', 'zip', deploy_dir)
print("✅ Created medicine_forecast_deployment.zip")

# Download
print("\n📥 Downloading...")
files.download('medicine_forecast_deployment.zip')

print("\n" + "="*80)
print("🎉 TRAINING COMPLETE!")
print("="*80)
print(f"\n✅ Models trained: {len(models)}")
print(f"✅ Forecasts generated: {len(all_forecasts)}")
print(f"✅ Deployment package ready!")
print("\n📦 Next Steps:")
print("   1. Extract medicine_forecast_deployment.zip")
print("   2. Upload to VPS: /home/user/health/")
print("   3. Run: ./deploy_models.sh")
print("   4. Test: python3 forecast_enhanced_gbr.py")
print("   5. View: http://your-server/forecast_dashboard.php")
print("\n" + "="*80)

## 🎉 Training Complete!

### What You Got:
- ✅ Trained Gradient Boosting models for each medicine
- ✅ Monthly predictions (1-12 months)
- ✅ Quarterly predictions
- ✅ Seasonal predictions (Philippine seasons)
- ✅ Model performance metrics
- ✅ Deployment package (ZIP)

### Next Steps:
1. Download the ZIP file
2. Upload to your VPS
3. Run deployment script
4. Start forecasting!

**📝 Note:** Retrain monthly as new data accumulates for best accuracy.

---

**🎓 For detailed documentation, see:** `FORECASTING_DOCUMENTATION.md`